In [1]:
# Exchange:Scrip name:strike size
# NSE: HDFCBANK: 10
# NSE: RELIANCE: 20
# NSE: SBIN: 5

In [2]:
import requests
import pandas as pd
import time
import xlwings as xw
from bs4 import BeautifulSoup
import datetime

In [3]:
TWO_PERCENT_MARKET_PRICE = 0.0

In [4]:
# get the last thursday of the current month
def last_thursday_version_2(year, month):
    
    if month == 1 or month == 2 or month == 3 or month == 4 or month == 5 or month == 6 or month == 7 or month == 8 or month == 9:
        date = f"{year}-0{month}-01"
    if month == 10 or month == 11 or month == 12:
        date = f"{year}-{month}-01"
        
    
    # we have a datetime series in our dataframe...
    df_Month = pd.to_datetime(date)
    
    # we can easily get the month's end date:
    df_mEnd = df_Month + pd.tseries.offsets.MonthEnd(1)
    
    # Thursday is weekday 3, so the offset for given weekday is
    offset = (df_mEnd.weekday() - 3) % 7

    # now to get the date of the last Thursday of the month, subtract it from 
    # month end date:
    df_Expiry = df_mEnd - pd.to_timedelta(offset, unit='D')

    return df_Expiry

In [5]:
# get the last thursday of the next month
def last_thursday(year, month):
    if month == 12:
        date = f"{year+1}-01-01"
    if month == 1 or month == 2 or month == 3 or month == 4 or month == 5 or month == 6 or month == 7 or month == 8:
        date = f"{year}-0{month+1}-01"
    if month == 9 or month == 10 or month == 11:
        date = f"{year}-{month+1}-01"
        
    
    # we have a datetime series in our dataframe...
    df_Month = pd.to_datetime(date)
    
    # we can easily get the month's end date:
    df_mEnd = df_Month + pd.tseries.offsets.MonthEnd(1)
    
    # Thursday is weekday 3, so the offset for given weekday is
    offset = (df_mEnd.weekday() - 3) % 7

    # now to get the date of the last Thursday of the month, subtract it from 
    # month end date:
    df_Expiry = df_mEnd - pd.to_timedelta(offset, unit='D')

    return df_Expiry

In [ ]:
ticker = "VEDL"
exchange = "NSE"

def current_market_price(ticker, exchange):
    
    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"

    for _ in range(1000000):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        class1 ="YMlKec fxKbKc"
        
        price = float(soup.find(class_=class1).text.strip()[1:].replace(",",""))
        yield price
        
        time.sleep(5)


# https://www.nseindia.com/api/option-chain-equities?symbol=VEDL
"""
headers = {"accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "en-US,en;q=0.9",
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

        }
"""
# https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY
"""
headers = {"accept-encoding": "gzip, deflate, br",
            "accept-language": "en-US,en;q=0.9",
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

        }
"""

while True:
    try:
        
        url = f"https://www.nseindia.com/api/option-chain-equities?symbol={ticker}"
        headers = {"accept-encoding": "gzip, deflate, br, zstd",
                   "accept-language": "en-US,en;q=0.9",
                   "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

        }

        session = requests.Session()
        data = session.get(url, headers=headers).json()["records"]["data"]
        ocdata = []
        for i in data:
            for j, k in i.items():
                if j == "CE" or j == "PE":
                    info = k
                    info["instrumentType"] = j
                    ocdata.append(info)

        df = pd.DataFrame(ocdata)
        wb = xw.Book("optionchaintracker.xlsx")
        st = wb.sheets("vedl")
        st.range("A1").value = df
        #print(df)
        
        for price in current_market_price(ticker, exchange):
            two_percent_cmp = price + 0.02 * price
            TWO_PERCENT_MARKET_PRICE = two_percent_cmp
            break
        
        print(TWO_PERCENT_MARKET_PRICE)
        
        
        # access dataframe for atm price
        atm = int(round(TWO_PERCENT_MARKET_PRICE/2.5,0)*2.5)
        print(atm)
        
        
        output_ce = pd.DataFrame()
        
        atm_pe = atm
        output_pe = pd.DataFrame()
        
        for _ in range(5):
            
            # (for ce)
            ab = True
            while ab:


                fd = df[df['strikePrice'] == atm]

                if fd.empty:
                    #print("empty df", atm)
                    atm = atm+0.5
                else:
                    ab = False

            #print(fd)
            
            # (for pe)
            ab_pe = True
            while ab_pe:


                fd_pe = df[df['strikePrice'] == atm_pe]

                if fd_pe.empty:
                    #print("empty df", atm_pe)
                    atm_pe = atm_pe-0.5
                else:
                    ab_pe = False
            
            #print(fd_pe)



            # (for ce)convert expiry date in particular format
            fd = fd.reset_index()
            for i in range(len(fd)):
                expiry_date_str = fd["expiryDate"].iloc[i]
                temp_expiry = datetime.datetime.strptime(expiry_date_str, '%d-%b-%Y')
                result_expiry = temp_expiry.strftime('%d-%m-%Y')
                fd.at[i, "expiryDate"]= result_expiry
            #print(fd)
            #print(type(fd["expiryDate"].iloc[0]))
            
            
            # (for pe) convert expiry date in particular format
            fd_pe = fd_pe.reset_index()
            for i in range(len(fd_pe)):
                expiry_date_str_pe = fd_pe["expiryDate"].iloc[i]
                temp_expiry_pe = datetime.datetime.strptime(expiry_date_str_pe, '%d-%b-%Y')
                result_expiry_pe = temp_expiry_pe.strftime('%d-%m-%Y')
                fd_pe.at[i, "expiryDate"]= result_expiry_pe

                
                
            # (last thursday (CE))
            today_year = datetime.datetime.now().year
            today_month = datetime.datetime.now().month
            today_day = datetime.datetime.now().day
            l_thursday = last_thursday_version_2(today_year, today_month)
            
            # if today is within the last thursday, then we will take current month last thursday as an adjusted_expiry
            if today_day < l_thursday.day or today_day == l_thursday.day and today_month == l_thursday.month:
                adjusted_expiry = last_thursday_version_2(today_year, today_month)
                print("in first if")
            # if today is greater than last thursday, then we will take next month last thursday as an adjusted_expiry
            if today_day > l_thursday.day and today_month == l_thursday.month:
                adjusted_expiry = last_thursday(today_year, today_month)
                print("in second if")
            
            format = '%d-%m-%Y'
            adjusted_expiry = adjusted_expiry.strftime(format)
            #year, month, day = str(l_thursday.date()).split("-")
            #l_thursday = f"{day}-{month}-{year}"
            #print(l_thursday)
            #print(l_thursday.date())
            #print(str(l_thursday.date()))
            #print(type(l_thursday.date()))
            #print(type(str(l_thursday.date())))
            
            # (last thursday (PE))
            today_year_pe = datetime.datetime.now().year
            today_month_pe = datetime.datetime.now().month
            today_day_pe = datetime.datetime.now().day
            l_thursday_pe = last_thursday_version_2(today_year_pe, today_month_pe)
            
            # if today is within the last thursday, then we will take current month last thursday as an adjusted_expiry
            if today_day_pe < l_thursday_pe.day or today_day_pe == l_thursday_pe.day and today_month_pe == l_thursday_pe.month:
                adjusted_expiry_pe = last_thursday_version_2(today_year_pe, today_month_pe)
                print("in first if")
            # if today is greater than last thursday, then we will take next month last thursday as an adjusted_expiry
            if today_day_pe > l_thursday_pe.day and today_month_pe == l_thursday_pe.month:
                adjusted_expiry_pe = last_thursday(today_year_pe, today_month_pe)
                print("in second if")
            
            format = '%d-%m-%Y'
            adjusted_expiry_pe = adjusted_expiry_pe.strftime(format)
            #year_pe, month_pe, day_pe = str(l_thursday_pe.date()).split("-")
            #l_thursday_pe = f"{day_pe}-{month_pe}-{year_pe}"
            
            
            # (subset_ce (CE))
            subset_ce = fd[(fd.instrumentType == "CE") & (fd.expiryDate == adjusted_expiry)]
            #print(subset_ce)
            output_ce = pd.concat([output_ce, subset_ce])
            
            
            # (subset_pe (PE))
            subset_pe = fd_pe[(fd_pe.instrumentType == "PE") & (fd_pe.expiryDate == adjusted_expiry_pe)]
            #print(subset_pe)
            output_pe = pd.concat([output_pe, subset_pe])
            
            #(for CE)
            atm += 2.5
            
            #(for PE)
            atm_pe -= 2.5
        
        print(output_ce)
        print(output_pe)
        
        time.sleep(300)
    except Exception as e:
        pass

274.635
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0           943                   228              31.888112   
0           138                   -17             -10.967742   
0          2023                   -58              -2.787122   
1           187                    -2              -1.058201   
1           657                    -6              -0.904977   

   totalTraded

275.04299999999995
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0           988                   273              38.181818   
0           138                   -17             -10.967742   
0          2024                   -57              -2.739068   
1           186                    -3              -1.587302   
1           655                    -8              -1.206637   

   

275.145
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0          1017                   302              42.237762   
0           140                   -15              -9.677419   
0          2023                   -58              -2.787122   
1           184                    -5              -2.645503   
1           657                    -6              -0.904977   

   totalTraded

275.4
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0          1025                   310              43.356643   
0           136                   -19             -12.258065   
0          2021                   -60              -2.883229   
1           183                    -6              -3.174603   
1           658                    -5              -0.754148   

   totalTradedVo

274.78799999999995
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0          1022                   307              42.937063   
0           138                   -17             -10.967742   
0          2022                   -59              -2.835175   
1           181                    -8              -4.232804   
1           658                    -5              -0.754148   

   

274.737
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0           996                   281              39.300699   
0           136                   -19             -12.258065   
0          2003                   -78              -3.748198   
1           181                    -8              -4.232804   
1           657                    -6              -0.904977   

   totalTraded

274.02299999999997
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0          1008                   293              40.979021   
0           136                   -19             -12.258065   
0          1988                   -93              -4.469005   
1           181                    -8              -4.232804   
1           667                     4               0.603318   

   

273.666
272
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
1     98        272.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE272.50   
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
1           172                   -21             -10.880829   
0          1013                   298              41.678322   
0           136                   -19             -12.258065   
0          1988                   -93              -4.469005   
1           181                    -8              -4.232804   

   totalTraded

274.635
275
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   index  strikePrice  expiryDate underlying                    identifier  \
0    101        275.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE275.00   
0    105        277.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE277.50   
0    109        280.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE280.00   
1    115        282.5  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE282.50   
1    119        285.0  28-03-2024       VEDL  OPTSTKVEDL28-03-2024CE285.00   

   openInterest  changeinOpenInterest  pchangeinOpenInterest  \
0          1013                   298              41.678322   
0           148                    -7              -4.516129   
0          1983                   -98              -4.709274   
1           181                    -8              -4.232804   
1           680                    17               2.564103   

   totalTraded

In [10]:
import tkinter as tk
from tkinter import ttk
import pandas as pd

def display_dataframe(df):
    root = tk.Tk()
    root.title("Pandas DataFrame in Tkinter")
    
    tree = ttk.Treeview(root)
    tree["columns"] = list(df.columns)
    
    for col in df.columns:
        tree.column(col, anchor="w")
        tree.heading(col, text=col, anchor='w')
    
    for i, row in df.iterrows():
        tree.insert("", "end", values=list(row))
    
    tree.pack(fill="both", expand=True)
    
    root.mainloop()

# Example DataFrame
data = {'Name': ['John', 'Anna', 'Peter', 'Linda'],
        'Age': [28, 25, 32, 35],
        'City': ['New York', 'Paris', 'London', 'Tokyo']}

df = pd.DataFrame(data)

# Call the function to display the DataFrame
display_dataframe(df)
